## Thư viện

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import re

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn import set_config
set_config(display='diagram')

In [2]:
# pyvi: https://pypi.org/project/pyvi/0.0.7.5/ - Vietnamese tokenizing tool
# !pip install pyvi
from pyvi import ViTokenizer, ViPosTagger

## Dữ liệu

Nguồn dữ liệu: VNFD Dataset - [vn_news_223_tdlfr.csv](https://github.com/thanhhocse96/vfnd-vietnamese-fake-news-datasets/blob/master/CSV/vn_news_223_tdlfr.csv)

Mô tả dữ liệu: [Mô tả tập VNFD](https://github.com/thanhhocse96/vfnd-vietnamese-fake-news-datasets/tree/master/CSV)

In [3]:
df = pd.read_csv("data/vn_news_223_tdlfr.csv")
df = df.drop(columns=['domain'])
df

,text,label
0,Thủ tướng Abe cúi đầu xin lỗi vì hành động phi...,1
1,Thủ tướng Nhật cúi đầu xin lỗi vì tinh thần ph...,1
2,Choáng! Cơ trưởng đeo khăn quàng quẩy banh nóc...,1
3,Chưa bao giờ nhạc Kpop lại dễ hát đến thế!!!\n...,1
4,"Đại học Hutech sẽ áp dụng cải cách ""Tiếq Việt""...",1
...,...,...
218,“Siêu máy bay” A350 sẽ chở CĐV Việt Nam đi Mal...,0
219,Thưởng 20.000 USD cho đội tuyển cờ vua Việt Na...,0
220,Trường Sơn giành HCV tại giải cờ vua đồng đội ...,0
221,Chuyện về chàng sinh viên Luật - Kiện tướng Lê...,0


Chúng ta sẽ chia dữ liệu thành 2 tập train và validation, với tỉ lệ 80/20.

Từ giờ, mọi bước liên quan đến tiền xử lý, trích xuất đặc trưng, train mô hình học máy đều chỉ thực hiện trên tập train. Tập validation sẽ được để dành cho việc kiểm tra lại mô hình.

In [4]:
X_df = df.drop("label", axis=1)
Y_sr = df["label"]

train_X_df, val_X_df, train_Y_sr, val_Y_sr = train_test_split(
    X_df, Y_sr, 
    test_size = 0.2, 
    stratify = Y_sr, 
    # random_state = 0
)

## Tiền xử lý văn bản tiếng Việt

Stopwords: https://github.com/stopwords/vietnamese-stopwords/blob/master/vietnamese-stopwords.txt

Tokenizer: Pyvi - Vietnamese tokenizing tool - https://pypi.org/project/pyvi/0.0.7.5/


Quá trình xử lý 1 đoạn text được thực hiện như sau:

* Tokenize
* Remove punctuations
* Remove special chars
* Remove links
* Lowercase
* Remove stopwords

In [5]:
with open("stopwords/vietnamese-stopwords.txt") as file:
    stopwords = file.readlines()
    stopwords = [word.rstrip() for word in stopwords]

punctuations = '''!()-–=[]{}“”‘’;:'"|\,<>./?@#$%^&*_~'''

special_chars = ['\n', '\t']

regex = re.compile(
        r'^(?:http|ftp)s?://' # http:// or https://
        r'(?:(?:[A-Z0-9](?:[A-Z0-9-]{0,61}[A-Z0-9])?\.)+(?:[A-Z]{2,6}\.?|[A-Z0-9-]{2,}\.?)|' # domain
        r'localhost|' # localhost
        r'\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3})' # ip
        r'(?::\d+)?' # port
        r'(?:/?|[/?]\S+)$', re.IGNORECASE)   

In [6]:
def tokenize(text):
    tokenized_text = ViPosTagger.postagging(ViTokenizer.tokenize(text))
    return tokenized_text[0]

def is_punctuation(token):
    global punctuations
    return True if token in punctuations else False

def is_special_chars(token):
    global special_chars
    return True if token in special_chars else False

def is_link(token):
    return re.match(regex, token) is not None

def lowercase(token):
    return token.lower()

def is_stopword(token):
    global stopwords
    return True if token in stopwords else False

# ===============================================================
# Process:
# Text -> Tokenize (pyvi) -> Remove punctuations -> Remove special chars 
# -> Remove links -> Lowercase -> Remove stopwords -> Final Tokens
# ===============================================================
def vietnamese_text_preprocessing(text):
    tokens = tokenize(text)
    tokens = [token for token in tokens if not is_punctuation(token)]
    tokens = [token for token in tokens if not is_special_chars(token)]
    tokens = [token for token in tokens if not is_link(token)]
    tokens = [lowercase(token) for token in tokens]
    tokens = [token for token in tokens if not is_stopword(token)]
    # return tokens
    return tokens

Ví dụ sử dụng:

In [7]:
# Trích 1 đoạn từ https://www.fit.hcmus.edu.vn/
demo_text = 'Trải qua hơn 25 năm hoạt động, Khoa Công nghệ Thông tin (CNTT) đã phát triển vững chắc và được Chính phủ bảo trợ để trở thành một trong những khoa CNTT đầu ngành trong hệ thống giáo dục đại học của Việt Nam.'

demo_text_to_tokens = vietnamese_text_preprocessing(demo_text)
print(demo_text_to_tokens)

['trải', '25', 'hoạt_động', 'khoa', 'công_nghệ', 'thông_tin', 'cntt', 'phát_triển', 'vững_chắc', 'chính_phủ', 'bảo_trợ', 'trở_thành', 'khoa', 'cntt', 'đầu', 'ngành', 'hệ_thống', 'giáo_dục', 'đại_học', 'việt_nam']


## Xây dựng mô hình máy học: MLPClassifier

#### Xây dựng pipeline

Pipeline: 
* **PreprocessAndFeaturesExtract**: Tiền xử lý văn bản và trích xuất đặc trưng
  * **Tiền xử lý văn bản**: Đã được cài đặt ở phần trên (hàm `vietnamese_text_preprocessing`)
  * **Trích xuất đặc trưng**: Văn bản -> Ma trận đặc trưng TF-IDF (`sklearn.feature_extraction.text.TfidfVectorizer` - https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html)
* **Mô hình phân lớp mạng neural MLPClassifier** (`sklearn.neural_network.MLPClassifier` - https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html)

In [8]:
class PreprocessAndFeaturesExtract(BaseEstimator, TransformerMixin):
    def __init__(self):
        # print("> PreprocessAndFeaturesExtract > INIT")
        self.trained_tokens = []

    # ===============================================================
    # Mỗi lần train chỉ gọi 1 lần fit
    # Mỗi lần fit thì sẽ tạo 1 list trained_tokens, bao gồm những tokens/đặc trưng đã được train
    # ===============================================================
    def fit(self, X_df, y=None):

        # Return a list of preprocessed_texts
        preprocessed_texts = []
        for index, row in X_df.iterrows():
            tokens = vietnamese_text_preprocessing(row['text'])
            preprocessed_texts.append(' '.join(tokens))
        preprocessed_texts = np.array(preprocessed_texts)

        # preprocessed_texts -> features
        tv = TfidfVectorizer(min_df = 0.0, max_df = 1.0, use_idf = True)
        tv_matrix = tv.fit_transform(preprocessed_texts)
        tv_matrix = tv_matrix.toarray()
        df_features = pd.DataFrame(np.round(tv_matrix, 2), columns = tv.get_feature_names_out())

        self.trained_tokens = df_features.columns.values

        # print(f"> PreprocessAndFeaturesExtract > FIT > X_df: {X_df.shape} > df_features: {df_features.shape}")
        # print(f"self.trained_tokens: {self.trained_tokens}")

        return self

    # ===============================================================
    # Khá giống phương thức fit (ở bước tiền xử lý, và trích xuất đặc trưng), tuy nhiên:
    # fit:       Gọi 1 lần duy nhất mỗi lần train
    # transform: Được gọi nhiều lần, và có thể áp dụng với nhiều X_df khác nhau (để tính score hay predict chẳng hạn),
    #            dựa trên cái model đã được train trước đó bằng fit
    # ===============================================================
    # fit tạo mới self.trained_tokens, transform thì không
    # ===============================================================
    # transform được cài đặt để trả về những đặc trưng ĐÃ được học,
    # những đặc trưng chưa học thì sẽ bỏ qua
    # ===============================================================
    def transform(self, X_df, y=None):

        # Return a list of preprocessed_texts
        preprocessed_texts = []
        for index, row in X_df.iterrows():
            tokens = vietnamese_text_preprocessing(row['text'])
            preprocessed_texts.append(' '.join(tokens))
        preprocessed_texts = np.array(preprocessed_texts)

        # Features Extraction
        # preprocessed_texts -> features
        # TF-IDF Model
        tv = TfidfVectorizer(min_df = 0.0, max_df = 1.0, use_idf = True)
        tv_matrix = tv.fit_transform(preprocessed_texts)
        tv_matrix = tv_matrix.toarray()
        vocab = tv.get_feature_names_out()
        temp_df_features = pd.DataFrame(np.round(tv_matrix, 2), columns=vocab)

        df_features = pd.DataFrame()

        for trained_token in self.trained_tokens:
            if trained_token in vocab:
                df_features[trained_token] = temp_df_features[trained_token]
            else:
                df_features[trained_token] = 0.000

        # print(f"\n> PreprocessAndFeaturesExtract > TRANSFORM > X_df: {X_df.shape} > df_features: {df_features.shape}")

        return df_features

In [9]:
# MLPClassifier
mlp_classifier = MLPClassifier(
    hidden_layer_sizes=(50),
    activation='relu',
    solver='lbfgs',
    random_state=0,
    max_iter=10000
)

# Pipeline: PreprocessAndFeaturesExtract -> MLPClassifier
pipeline_mlp = Pipeline(
    steps=[
           ("vnpreprocess", PreprocessAndFeaturesExtract()),
           ("mlpclassifier", mlp_classifier)
           ]
)
pipeline_mlp

Pipeline(steps=[('vnpreprocess', PreprocessAndFeaturesExtract()),
                ('mlpclassifier',
                 MLPClassifier(hidden_layer_sizes=50, max_iter=10000,
                               random_state=0, solver='lbfgs'))])

#### Train

Tiến hành huấn luyện mô hình trên tập train

In [10]:
pipeline_mlp.fit(train_X_df, train_Y_sr)
print("> Train completed")

> Train completed


#### Score

Tính accuracy của mô hình, sử dụng lần lượt tập train và tập validation.

In [11]:
train_acc = pipeline_mlp.score(train_X_df, train_Y_sr)
print(f"Train Accuracy = {train_acc}")

val_acc = pipeline_mlp.score(val_X_df, val_Y_sr)
print(f"Validation Accuracy = {val_acc}")

Train Accuracy = 1.0
Validation Accuracy = 0.8888888888888888


#### Predict

Dự đoán label của văn bản mới

In [12]:
# # Nếu có 1 tập muốn predict, tên "predict_X_df", thì predict tập đó như sau:
# pred_results = pipeline_mlp.predict(predict_X_df)
# pred_results